In [2]:
from cryptography.fernet import Fernet
from datetime import datetime
import json
import random
import yagmail

In [3]:
def generate_constraints(santa_list_with_details):
    constraints = []
    santa_names = [x[0] for x in santa_list_with_details]
    for santa_id, (_, _, santa_so) in enumerate(santa_list_with_details):
        if santa_so is None:
            continue
        santa_so_id = santa_names.index(santa_so) # will throw a ValueError if santa_so is not in santa_names
        constraints.append((santa_id, santa_so_id))
    return constraints

In [4]:
def generate_permutation(n):
    return random.sample(range(n), n)

In [5]:
def is_mapping(permutation, constraints):
    """Determine if a permutation is a valid mapping of santas to santees.
    
    Valid mapping needs to meet two sets of conditions:
    i) a santa cannot give to himself/herself
    ii) a santa cannot give to his/her significant other (constraints)
    """

    n = len(permutation)
    
    # check i) cannot give to self
    for i in range(n):
        if permutation[i] == i:
            return False 
    
    # check ii) cannot give to SO
    for i, j in constraints:
        if permutation[i] == j:
            return False 
    
    return True # otherwise, valid

In [6]:
def generate_mapping(n, constraints, max_iter=10000):

    # initialize to identity
    permutation = list(range(n))
    counter = 0
    
    # keep regenerating permutations until we get a valid mapping
    while not is_mapping(permutation, constraints):
        if counter > max_iter:
            raise Exception("Failed to generate a mapping after {} iterations".format(max_iter))
        
        permutation = generate_permutation(n)
        counter += 1
    
    print(f"Valid mapping generated after {counter} attempts")
    return permutation

In [7]:
def send_email(santa_name, 
               santa_email, 
               santee, 
               sender, 
               password, 
               year,
               budget=20, 
               debug_mode=True):

    if debug_mode:
        santa_email = sender

    yag = yagmail.SMTP(sender, password=password)
    yag.send(
        to=santa_email,
        subject=f"Secret Santa {year}! 🎄🎅",
        contents=f"""Ahoj {santa_name},

Tento rok si Secret Santa pre: <strong>{santee}</strong>.

Pšt, nikomu to nehovor!

<small>Budget: {budget}€. Túto správu poslal SecretSantaBot.</small>""")

    print(f"[{datetime.now()}] email sucessfully sent to '{santa_name}' <{santa_email}>")

In [17]:
current_year = datetime.today().year

# read config
with open("config-example.json", "r") as f:
    santa_list_with_details = json.load(f)

# extract constraints
constraints = generate_constraints(santa_list_with_details)

# generate encryption key for writing to disk
key = Fernet.generate_key()
fernet = Fernet(key)

# init the dictionary to be written to disk
santa_dict = {}
santa_dict['_fernet_key'] = key.decode("utf8")

# generate mapping
santa_to_santee_mapping = generate_mapping(n=len(santa_list_with_details), constraints=constraints)

# loop over all santas
for santee_id, (santa_name, santa_email, santa_so) in zip(santa_to_santee_mapping, 
                                                   santa_list_with_details):
    # get santee name from id
    santee_name = santa_list_with_details[santee_id][0]

    # encrypt santee name and store it in the dict
    santa_dict[santa_name] = fernet.encrypt(santee_name.encode()).decode()

    # send email to santa
    send_email(santa_name=santa_name, 
               santa_email=santa_email, 
               santee=santee_name,
               sender="my.email@gmail.com", 
               password="VerySecretPassword",
               year=current_year,
               budget=20, 
               debug_mode=True)

    # debug print - comment when running for real
    print(f"{santa_name} --> {santee_name} [constraint: {santa_so}]")

# export file with encrypted santees (backup)
with open(f"secret-santa-{current_year}-encrypted.json", "w") as f:
    json.dump(santa_dict, f, sort_keys=True, indent=4)

Valid mapping generated after 1 attempts
Person1 --> Person8 [constraint: Person2]
Person2 --> Person7 [constraint: Person1]
Person3 --> Person2 [constraint: Person4]
Person4 --> Person6 [constraint: Person3]
Person5 --> Person1 [constraint: Person6]
Person6 --> Person3 [constraint: Person5]
Person7 --> Person4 [constraint: None]
Person8 --> Person5 [constraint: None]
